In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sapy import Clock, ProgramCounter, MemoryAddressRegister, \
    RandomAccessMemory, SwitchBoard, RegisterA, RegisterB, RegisterOutput, \
    ArithmeticUnit, RegisterInstruction, Computer


# Hex and Binary Literals in Python

In [3]:
0xF

15

In [4]:
0xFF

255

In [5]:
print(15_000) # underscore as separators are allowed
print(0b1111_1111)

15000
255


You can also format integers to HEX using X

In [6]:
f"{132:X}"

'84'

# Clocking Signals

In [7]:
clk = Clock()
class DummyComponent():
    def data(self, **kwargs):
        print("data accessed")
        return 42
    def clock(self, **kwargs):
        print("clock stepped")

dc = DummyComponent()
clk.add_component(dc)

In [8]:
print("step 1: ---------")
clk.step()
print("step 2: ---------")
clk.step()

step 1: ---------
data accessed
clock stepped
step 2: ---------
data accessed
clock stepped


# Program Counter

To enable a component to take an action on a clock, you pass in the correct control bit as a keyword argument. To enable the PC to output to the data but, use the "ep" control bit.

In [9]:
pc = ProgramCounter()

In [10]:
pc.clock()
pc.data()

In [11]:
pc.clock()
pc.data(ep=True)

0

In [12]:
pc.clock()
pc.data(ep=True)

0

To increment the program counter, the control bit is "cp".

In [13]:
pc.clock(cp=True)
pc.data(ep=True)

1

In [14]:
pc.clock(cp=True)
pc.data(ep=True)

2

In [15]:
pc.clock(cp=True)
pc.data(ep=True)

3

For jump instructions you set the program counter directly. This is done with the "lp" control bit.

In [16]:
pc.clock(data=0xD, lp=True)
print(f"{pc.data(ep=True):X}")
print(f"{pc.data(ep=True)}")

D
13


# Accumulator (Register A)

In [17]:
reg_a = RegisterA()
print(f"The register value on reset is: {reg_a.data(ea=True):X}")

reg_a.clock(data=0xAB, la=True)
reg_a.data(ea=True)
print(f"The register has latched a value: {reg_a.data(ea=True):X}")


The register value on reset is: 0
The register has latched a value: AB


# Adder

Some components have extra attributes that allow it to communicate directly to other components outside of the bus. 

The adder is not clocked, it doesn't even have state. The data output from the adder is alway the instantaneous sum (or difference) of Reg A + Reg B

In [18]:
reg_a = RegisterA()
reg_a.clock(data=4, la=True)

reg_b = RegisterB()
reg_b.clock(data=3, lb=True)

adder = ArithmeticUnit(reg_a, reg_b)

assert adder.data() is None
adder.data(eu=True)


7

In [19]:
reg_b.clock(data=1, lb=True)
adder.data(eu=True)

5

Subtraction

In [20]:
adder.data(eu=True, su=True)

3

# Memory

Ram is another example of a system that has communication outside the bus, it always access the RAM value stored at the address latched in the Memory Address Register.

In [21]:
mar = MemoryAddressRegister()
ram = RandomAccessMemory(mar)

# store address for ram in register
mar.clock(data=0xF, lm=True)
print(f"Memory Address: {mar.address():X}")

# clock data into ram at the address set above
ram.clock(data=0xAB, lr=True)
assert ram.data() == None
print(f"Memory at address: {mar.address():X} is {ram.data(er=True):X}") # should be 0xAB
print(f"Memory at address: {mar.address()} is {ram.data(er=True)}") # just to demystfy hex..

Memory Address: F
Memory at address: F is AB
Memory at address: 15 is 171


# Controller: Sequencer + Decoder + Clock

In [22]:
reg_i = RegisterInstruction()
instruction = 0xFF # both opcode and argument, 8bits
reg_i.clock(data=instruction, li=True)


In [23]:
print(f"The opcode is {reg_i.opcode():X}")

The full store instruction is FF
The opcode is FF


# Example Sequencing: LDA

In [28]:
from sapy import Computer

cpu = Computer()
rom = [
    0x00, 0x09, # 0x0 LDA 09H
    ]
cpu.switches.load_program(rom)

for _ in range(7):
    cpu.step(debug=True) 

------------------------------------------
T1: Data: 0, Control Word: {'ep': True, 'lm': True}
T2: Data: None, Control Word: {'cp': True}
T3: Data: 0, Control Word: {'er': True, 'li': True}
T4: Data: 1, Control Word: {'ep': True, 'lm': True, 'cp': True}
T5: Data: 9, Control Word: {'er': True, 'lm': True}
T6: Data: 0, Control Word: {'er': True, 'la': True}
T7: Data: None, Control Word: {}


In [29]:
cpu.step(debug=True)

------------------------------------------
T1: Data: 2, Control Word: {'ep': True, 'lm': True}


# Output (Register O)

In [30]:
reg_o = RegisterOutput()
reg_o.clock(data=0x90, lo=True)


Output Display: 90


In [31]:
# monkey patch output function for fancy display
def display_number(x):
    print(f"***### {x} ###***")
reg_o.output_function = display_number

In [32]:
reg_o.clock(data=0x90, lo=True)

***### 144 ###***


# Full Computer with ROM

In [33]:
0x04import time

In [40]:
cpu = Computer()
rom = [
    0x00, 0x07, # 0x00 LDA 07H
    0x01, 0x08, # 0x02 ADD 08H
    0x03,       # 0x04 OUT
    0x04, 0x02, # 0x05 JMP 02H
    0x00,       # 0x07 A1H
    0x03,       # 0x08 22H
    ]
cpu.switches.load_program(rom)

In [41]:
for _ in range(9):
    cpu.step(instructionwise=True)

Output Display: 3
Output Display: 6
Output Display: 9


In [42]:
for _ in range(3):
    cpu.step(instructionwise=True, debug=True)

------------------------------------------
T1: Data: 5, Control Word: {'ep': True, 'lm': True}
T2: Data: None, Control Word: {'cp': True}
T3: Data: 4, Control Word: {'er': True, 'li': True}
T4: Data: 6, Control Word: {'ep': True, 'lm': True, 'cp': True}
T5: Data: 2, Control Word: {'er': True, 'lp': True}
T6: Data: None, Control Word: {}
T7: Data: None, Control Word: {}
------------------------------------------
T1: Data: 2, Control Word: {'ep': True, 'lm': True}
T2: Data: None, Control Word: {'cp': True}
T3: Data: 1, Control Word: {'er': True, 'li': True}
T4: Data: 3, Control Word: {'ep': True, 'lm': True, 'cp': True}
T5: Data: 8, Control Word: {'er': True, 'lm': True}
T6: Data: 3, Control Word: {'er': True, 'lb': True}
T7: Data: 12, Control Word: {'eu': True, 'la': True}
------------------------------------------
T1: Data: 4, Control Word: {'ep': True, 'lm': True}
T2: Data: None, Control Word: {'cp': True}
T3: Data: 3, Control Word: {'er': True, 'li': True}
T4: Data: 12, Control Word:

In [43]:
import ipywidgets as widgets
from IPython.display import display
gui_output_display = widgets.FloatText()
display(gui_output_display)

# monkey patch output function
def display_number(x):
    gui_output_display.value = x

cpu.reg_o.output_function = display_number

FloatText(value=0.0)

In [44]:
for _ in range(3):
    time.sleep(.2)
    cpu.step(instructionwise=True, debug=False)

In [45]:
for _ in range(27):
    time.sleep(.03)
    cpu.step(instructionwise=True, debug=False)

In [46]:
while True:
    time.sleep(.03)
    cpu.step(instructionwise=True, debug=False)

KeyboardInterrupt: 